In [3]:
#CAISO TODAY'S OUTLOOK SCRAPER
#=================================================================================
#DOWNLOADS DAILY 5-MIN DATA FOR A SPECIFIED DATE RANGE AND DATASET FROM CAISO'S "TODAY'S OUTLOOK" PAGE.
#SAVES TO path/CAISOcsv/Results, WHERE "path" IS THE DIRECTORY WHERE THIS FILE IS SAVED

#The following libraries must be installed:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import os
import glob
import pandas as pd
import shutil
import time
import sys
import numpy as np


#INPUTS
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url = 'https://www.caiso.com/TodaysOutlook/Pages/index.html'
date_xpath = '//*[@id="section-demand-trend"]/div[4]/div/div/div/nav/div[1]/input'
dropdown_xpath = '/html/body/div[1]/div[3]/div[6]/div[4]/div/div/div/nav/div[3]/button'
download_xpath = '/html/body/div[1]/div[3]/div[6]/div[4]/div/div/div/nav/div[3]/div/a'

filename = 'netdemand_test2.csv'

#date format: datetime.date(year,month,day)
startdate = datetime.date(2023,4,1)
enddate = datetime.date(2023,4,30)

#Set headless to 'yes' to hide browser.
headless = 'yes'
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#Must have Chrome browser.
#Must have corresponding version of chromedriver installed here (or change path below):
chromedriver = '/usr/local/bin/chromedriver'

#INSTRUCTIONS:
#Update Inputs
#Click 'Save'
#Click 'Run' (or hit SHIFT-ENTER)

#NOTE:
#Some data cleanup will probably be required, for example due to errors in CAISO's data.
#See error report in terminal for which dates need to be fixed.
#Some common errors:
    #Date has 289 values instead of 288.
        #Sometimes CAISO's data for a given date includes an extra value at the end, corresponding to the following day's first value.
        #This appears in the results file as 2 "0:00" values.
        #See "dates with missing or extra data" list returned in terminal.
        #Fix: The code should fix this itself, but if it doesn't, find the imposter value (probably a 0:00). If one of the 2 0:00 values matches the 0:00 value from the following date, delete that value.
        #If it's unclear which value is wrong, download the date's CSV manually from Today's Outlook (or just hover over the chart). That CSV will still be in the original order, so the imposter 0:00 value will appear at the end.
    #Duplication. If the code tries to download a date's data before it finishes loading, the previous date's data will be duplicated.
        #See "duplicate_data_log:" list returned in terminal
        #Fix: Download the duplicate date individually, or run code again for that range of dates. Replace duplicate data with newly downloaded data.

#=================================================================================
#SETUP DIRECTORIES AND CHROMEDRIVER

#Make CAISOcsv & Results folders if they don't already exist
#Clear and make temporary directories for downloads and intermediary transposed csv's
#TempDir1 for downloads, TempDir2 for intermiediary transposed csv's, Results for final csv's:
tic = time.time()
date = startdate
path = sys.path[0]
CAISOcsv_path = f'{path}/CAISOcsv/'
Results_path = f'{path}/CAISOcsv/Results/'
TempDir1_path = f'{path}/CAISOcsv/TempDir1/'
TempDir2_path = f'{path}/CAISOcsv/TempDir2/'
combined_csv_path = f'{path}/CAISOcsv/TempDir2/combined_csv.csv'

try:
    os.mkdir(CAISOcsv_path)
    print("made new CAISOcsv folder")
except FileExistsError:
    pass

try:
    os.mkdir(Results_path)
    print("made new Results folder")
except FileExistsError:
    pass

try:
    os.mkdir(TempDir1_path)
except FileExistsError:
    try:
        shutil.rmtree(TempDir1_path)
        os.mkdir(TempDir1_path)
        print("made new TempDir1")
    except OSError as e:
        print("Error: %s : %s" %(TempDir1_path, e.strerror))

try:
    os.mkdir(TempDir2_path)
except FileExistsError:
    try:
        shutil.rmtree(TempDir2_path)
        os.mkdir(TempDir2_path)
        print("made new TempDir2")
    except OSError as e:
        print("Error: %s : %s" %(TempDir2_path, e.strerror))

#initiate chromedriver, set download directory to TempDir1, make headless, get CAISO page.

options = webdriver.ChromeOptions()
prefs = {"download.default_directory" : TempDir1_path}
options.add_experimental_option("prefs",prefs)
if headless == 'yes':
    options.add_argument('headless')
browser = webdriver.Chrome(executable_path=chromedriver, options=options) #DeprecationWarning: use options instead of chrome_options // browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)
browser.get(url)

#=================================================================================
#DOWNLOADER

wait = WebDriverWait(browser, 10)

try:
    element = wait.until(EC.element_to_be_clickable((By.XPATH, date_xpath)))
except:
    print('Could not find date field')
'''
for element in elements:
    if 'batteries-date' in element.get_attribute('class'):       
'''        
#Enters date
while date <= enddate:
    element.click()
    element.clear()
    date_input = (date.strftime("%m/%d/%Y"))
    element.send_keys(date_input)
    element.send_keys(Keys.ENTER)

    #Clicks download link
    button1 = browser.find_element(By.XPATH, dropdown_xpath)
    time.sleep(1.5)       #make sure it has time to load. Otherwise, there can be missing values or duplicates. I TRIED CHANGING 1.5 TO 2, BUT IT DIDN'T REDUCE DUPLICATES.
    button1.click()
    button2 = browser.find_element(By.XPATH, download_xpath)

    #in case the browser is lagging, it will keep trying to click the link
    while True:
        try:
            button2.click()
            print("downloading", date_input)
        except:     #add specific exceptions?
            time.sleep(0.5)
            print("needed a nap")
            continue
        break
    date = date + datetime.timedelta(1)

#kill invisiable headless browser
time.sleep(2)        #make sure the last date has time to download.
browser.close()

#=================================================================================
#FORMAT DAILY DATA
def autotrim(data, date):
    print(f'autotrimming {date} because it has {len(data.columns)} values.')
    if data.iloc[0,288] == '00:00' or data.iloc[0,288] == '0:00':
        data.drop(data.columns[[288]], axis=1, inplace=True)
        print(f'Fixed {date}. Now has {len(data.columns)} values.')
    else:
        print(f'{date} still 289 values. Check manually.')
    return(data)

#grabbing csv's from TempDir1, transposing, stripping headings, adding date column
os.chdir(TempDir1_path)
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]            
print("{} files in all_filenames".format(len(all_filenames)))
print("formatting data...")
missing_data_log = []

for f in range(0,len(all_filenames)):
    data = pd.read_csv(TempDir1_path+all_filenames[f], header=None, index_col=0)
    
    #Cell A1 (data[0][0]) contains the date, and sometimes other text
    date_raw = (data.iloc[0].name)
    #Extract the date from the above string by finding the positions of the first and last digits in the string.
    date_pos = [i for i,c in enumerate(date_raw) if c.isdigit()]
    date = date_raw[min(date_pos):max(date_pos)+1]
    print("formatting", date)
    
    #fix the double-0:00 issue
    if len(data.columns) == 289:
        autotrim(data, date)

    #transpose & creating date column
    df = data.T
    date_column = [date]*len(df)
  
    #flag dates with missing or extra data
    if len(df) != 288:
        print("warning: {} has {} elements".format(date,len(df)))
        missing_data_log.append(date)
    
    #add date column & save csv to TempDir2
    df.insert(0,"Date", date_column,True)
    df.rename(columns={date_raw: 'Time'}, inplace = True)
    df.to_csv(TempDir2_path+all_filenames[f])    

    
#=================================================================================
#CONCATENATOR+SORTER

os.chdir(TempDir2_path)
day_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
print("concatenating...")
combined_csv = pd.concat([pd.read_csv(f) for f in day_filenames])

#create datetime object for sorting
print("combined_csv")
combined_csv.to_csv(TempDir2_path+'combined_csv.csv')
print("saved combined csv")

#Add datetime column
#I added Try/Except because one value in CAISO's import data was missing a timestamp, which caused an error when I tried to strip the time, but it was still able to make a datetime object without stripping.
print("adding datetime for sorting...")
df = pd.read_csv(combined_csv_path)
df = df.drop('Unnamed: 0',axis=1)
date_error_counter = 0
time_error_counter = 0
for index,row in df.iterrows():
    try:
        DATE = df.loc[index,'Date'].strip()
    except Exception as e:
        date_message = str(e)
        print (f"Failed to strip Date for index: {index}, row: {row}. Error message: {date_message}")
        date_error_counter += 1
    try:
        TIME = df.loc[index,'Time'].strip()
    except Exception as e:
        time_message = str(e)
        print (f"Failed to strip Time for index: {index}, row: {row}. Error message: {time_message}")
        time_error_counter += 1
    df.loc[index,'Datetime'] = datetime.datetime.combine(datetime.datetime.strptime(DATE, '%m/%d/%Y') ,datetime.datetime.strptime(TIME,'%H:%M').time())
try:
    print (f"Failed to strip Date for {date_error_counter} rows. Error message: {date_message}")
except:
    pass
try:
    print (f"Failed to strip Time for {time_error_counter} rows. Error message: {time_message}")
except:
    pass

#sort by datetime
df = df.drop('Unnamed: 0.1', axis=1)
print("sorting")
df = df.sort_values(by='Datetime')
final_df = df.reset_index(drop=True)

#=================================================================================
#REPORT + SAVE

#report missing data
print("-------------------------")
print("dates queried:", enddate - startdate + datetime.timedelta(1))
print("dates returned:", len(all_filenames))
print("Given dates returned:")
print("  values expected:", 288*len(all_filenames))
print("  values returned:", len(final_df))
print("dates with missing or extra data:", missing_data_log)
print("-------------------------")

#save to Results folder
final_df.to_csv(Results_path+filename)
print("saved final csv")


#=================================================================================
#DUPLICATE CHECK

print("checking for duplicates...")
df = pd.read_csv(Results_path+filename)  ###Untested###
df['Date'] = df['Date'].str.strip()
df.set_index("Date", inplace=True)
df.head()
date = startdate
duplicate_data_log = []
previous = []
notfirstday = 0

#Use values in second-to-last column for Duplicate Check
value_col = df.columns[-2]
print(f'Value column used in Duplicate Check: {value_col}')

while date <= enddate:
    date_input = (date.strftime("%m/%d/%Y"))
    elements = df.loc[[date_input],[value_col]]   ###Untested###
    if notfirstday == 0:
        pass
    elif np.array_equal(elements.values,previous.values):
        print("warning: {} is a duplicate of the previous date".format(date_input))
        duplicate_data_log.append(date_input)
    previous = elements
    date = date + datetime.timedelta(1)
    notfirstday = 1
print("duplicate_data_log:",duplicate_data_log)
toc = time.time()
print(toc-tic,'seconds elapsed')
print("DONE")

made new TempDir1
made new TempDir2


/var/folders/qp/2qqrxy2d4vs58h_j13bl4_zc0000gn/T/ipykernel_70318/202624430.py:113: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chromedriver, options=options) #DeprecationWarning: use options instead of chrome_options // browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)


downloading 04/01/2023
downloading 04/02/2023
downloading 04/03/2023
downloading 04/04/2023
downloading 04/05/2023
downloading 04/06/2023
downloading 04/07/2023
downloading 04/08/2023
downloading 04/09/2023
downloading 04/10/2023
downloading 04/11/2023
downloading 04/12/2023
downloading 04/13/2023
downloading 04/14/2023
downloading 04/15/2023
downloading 04/16/2023
downloading 04/17/2023
downloading 04/18/2023
downloading 04/19/2023
downloading 04/20/2023
downloading 04/21/2023
downloading 04/22/2023
downloading 04/23/2023
downloading 04/24/2023
downloading 04/25/2023
downloading 04/26/2023
downloading 04/27/2023
downloading 04/28/2023
downloading 04/29/2023
downloading 04/30/2023
30 files in all_filenames
formatting data...
formatting 04/13/2023
autotrimming 04/13/2023 because it has 288 values.
Fixed 04/13/2023. Now has 288 values.
formatting 04/07/2023
autotrimming 04/07/2023 because it has 288 values.
Fixed 04/07/2023. Now has 288 values.
formatting 04/06/2023
autotrimming 04/06/20